In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from pandas import DataFrame


%matplotlib inline
plt.style.use('bmh')


#Datasets#
ds16= pd.read_csv('Downloads/raw/Estudiante_5-6 año Secundaria 2016.csv', sep= ',')
dict_2016 = pd.read_excel('Downloads/raw/Diccionarios/Dic_2016_Sec.xlsx')

#Bases Anexas#

pob_escuelas = pd.read_csv('Downloads/gobiernoslocales_2020/poblacion.csv', sep=',')
q_escuelas = pd.read_csv('Downloads/gobiernoslocales_2020/escuelas.csv', sep=',')
dpto_arg = pd.read_csv('Downloads/gobiernoslocales_2020/departamentos.csv', sep=',')
mun = pd.read_csv('Downloads/gobiernoslocales_2020/ign_municipio.csv', sep=';')
dptos = pd.read_csv('Downloads/gobiernoslocales_2020/ign_departamento.csv', sep=';')
dtos_match = pd.read_csv('Downloads/gobiernoslocales_2020/Dpto_transformado.csv', sep=';')
dptos_merge = pd.read_csv('Downloads/Dptos BsAs Mod.csv', sep= ';')

print("dataset total",ds16.shape)
print("dataset total", ds16.size)
print("diccionario",dict_2016.shape)
print("diccionario", dict_2016.size)

#Muestra BsAs y CABA#
val_filt = [2, 6]
df = ds16[ds16.cod_provincia.isin(val_filt)]
df = df.infer_objects()

print("muestra", df.shape)
print("muestra", df.size)


#Se agrega la información geográfica del Municipio#
df2 = df.merge(dptos_merge, on =['Municipio'], how ='left',indicator = True) 
df2.size

In [ ]:
#Parametrización Gráfico#
def plot_multiple_vars(df,x,y,title='Titulo de referencia',label= False):
    
    data =df[y].value_counts().to_frame().reset_index().sort_values(y)
    
    if list(label):
        label = label[label.Variable==y].set_index('Códigos')
        data['index'] =data['index'].map(label['Codigo_Et']) 
        
    plt.figure(figsize=(20,10))
    
    sns.set(style="whitegrid")
    ax = sns.barplot(data = data , 
                     x =x,y=y,order = data['index']
                    )



    ax.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))
    
    if list(label):
        ax.set(xlabel= label['Variabl_Et'].unique().item(), ylabel='Count')
    else:
        ax.set(xlabel= y, ylabel='Count')
    # add proper Dim values as x labels
    for item in ax.get_xticklabels(): item.set_rotation(90)
    for i, v in enumerate(data[y].iteritems()):        
        ax.text(i ,v[1], "{:,}".format(v[1]), color='m', va ='bottom', rotation=45)
    plt.tight_layout()
    plt.title(title,size =20)
    return plt.show()

variables_explicitas =dict_2016[dict_2016.Variable.isin(ds16.columns)]
variables_explicitas.head()
len(variables_explicitas.Variable.unique())

In [ ]:
#Estadística Descriptiva y Correlaciones#
desc = df2.describe(include = 'all').transpose()
export_excel2= desc.to_excel('Downloads/Outputs/Summ_First_sinmuni.xlsx',index= True, header=True)
corr = df2.corr(method = 'spearman')
export_excel2= corr.to_excel('Downloads/Outputs/Corr_0310.xlsx',index= True, header=True)

In [ ]:
missing_ratio = df2.isnull().sum().sum() / df2.size
null_ratio = df2.isin([-9.0, -1.0,""]).sum().sum() / df2.size
print("missing ratio", round(missing_ratio,2))
print("null ratio", round(null_ratio,2))

In [ ]:
#Plot Variables por Categoría#
plot = pd.crosstab(index=m['isocioa'],
            columns=m['ldesemp']
                  ).apply(lambda r: r/r.sum() *100,
                          axis=0).plot(kind='bar', stacked=True)

In [ ]:
#Plot Variables Pie Chart#
plot = df2['ldesemp'].value_counts().plot(kind='pie', autopct='%.2f', 
                                            figsize=(6, 6),


In [ ]:
#Gráfico por variable#
plot_multiple_vars(df2, x ='index',y='Ap48a',title='¿Con qué frecuencia usas la computadora para trabajar en clase de Informática?',label = dict_2016)

In [ ]:
#Gráfico más de una variable#
var = 'sector'
ap = 'Subcategory'

variables_explicitas.loc[variables_explicitas.Variable == 'cod_provincia']
secundario_2016_sector_provincia = df[['cod_provincia',var]].copy()

secundario_2016_sector_provincia['cod_provinca'] = secundario_2016_sector_provincia['cod_provincia']\
                                .map(variables_explicitas.loc[variables_explicitas.Variable == 'cod_provincia']\
                                .set_index('Códigos')['Codigo_Et'])

secundario_2016_sector_provincia[var] = secundario_2016_sector_provincia[var]\
                                .map(variables_explicitas.loc[variables_explicitas.Variable == var]\
                                .set_index('Códigos')['Codigo_Et'])

secundario_2016_sector_provincia['count'] = 1

groupby_secundario_2016_sector_provincia = secundario_2016_sector_provincia\
                                            .groupby(['cod_provincia',var]).count().reset_index()
    
groupby_secundario_2016_sector_provincia.groupby(var).sum()['count']

#
data = (groupby_secundario_2016_sector_provincia.set_index([var,
                                                     'cod_provincia'])['count'] / groupby_secundario_2016_sector_provincia.groupby('cod_provincia').sum()['count'])\
.to_frame().reset_index()


#
data= groupby_secundario_2016_sector_provincia.pivot(index=var,columns='cod_provincia',values='count')
data.plot(kind='bar',figsize=(20,10), label=dict_2016)

In [ ]:
#Tablas#
from tabulate import tabulate

preg = 'sector'
filt1 = ['CABA']
filt2 = ['AMBA sin CABA']
filt3 = ['Interior GBA']
caba = df2[df2.Subcategory.isin(filt1)]
gba = df2[df2.Subcategory.isin(filt2)]
interior = df2[df2.Subcategory.isin(filt3)]

privado_caba = caba.sector.value_counts()[2.0]
privado_gba = gba.sector.value_counts()[2.0]
privado_int = interior.sector.value_counts()[2.0]
pub_caba = caba.sector.value_counts()[1.0]
pub_gba = gba.sector.value_counts()[1.0]
pub_int = interior.sector.value_counts()[1.0]
m1 = privado_caba + pub_caba + privado_gba + pub_gba
m3 = privado_int + pub_int
mtot = privado_caba + pub_caba + privado_gba + pub_gba + privado_int + pub_int


data = [("Privado AMBA", (privado_caba + privado_gba) / m1),
       ("Privado Int GBA", (privado_int) / m3),
       ("Publico AMBA", (pub_caba+ pub_gba) / m1),
       ("Publico Int GBA", (pub_int)/m3)] 

dataset= tabulate(data)

print(dataset)

In [ ]:
#Feature Importance#

#Elimino Variables que no necesito#
df3 = df2.copy()
delete = pd.read_csv('Downloads/elimina.csv', sep= ';')
col_list = delete['del']
df3.drop(col_list,axis=True,inplace=True)
df3.shape

#Separo Target y Variables Predictoras#
df3 = df3.replace('nan','NaN')
df3 = df3.fillna(0)
data = df3.dropna()
features = [feat for feat in list(data) 
            if feat != 'ldesemp']

datamat = np.array(features)
X, y = data[features], data.ldesemp

In [ ]:
#Random Forest#
import sklearn
print(sklearn.__version__)

from collections import defaultdict
from scipy.stats import spearmanr
from scipy.cluster import hierarchy

from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size= 0.1)

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
print("Accuracy on test data: {:.2f}".format(clf.score(X_test, y_test)))

result = permutation_importance(clf, X_train, y_train, n_repeats=10, random_state=42) 
perm_sorted_idx = result.importances_mean.argsort()

tree_importance_sorted_idx = np.argsort(clf.feature_importances_) 
tree_indices = np.arange(0, len(clf.feature_importances_)) + 0.5

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(50, 24))
ax1.barh(tree_indices,
         clf.feature_importances_[tree_importance_sorted_idx], height=0.7)
ax1.set_yticklabels(datamat[tree_importance_sorted_idx])
ax1.set_yticks(tree_indices)
ax1.set_ylim((0, len(clf.feature_importances_)))
ax2.boxplot(result.importances[perm_sorted_idx].T, vert=False,
            labels=datamat[perm_sorted_idx])
fig.tight_layout()
plt.show()

In [ ]:
#XGBOOST#
from numpy import loadtxt
from xgboost import XGBClassifier, plot_importance

model = XGBClassifier()
model.fit(X, y)

# feature importance
print(model.feature_importances_)

plot_importance(model)
pyplot.show()

In [ ]:
from numpy import sort
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel


# define custom class to fix bug in xgboost 1.0.2
class MyXGBClassifier(XGBClassifier):
               @property
               def coef_(self):
                              return None

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=7)

model = MyXGBClassifier()
model.fit(X_train, y_train)

# make predictions for test data and evaluate
predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

# Fit model using each importance as a threshold
thresholds = sort(model.feature_importances_)
for thresh in thresholds:
    # select features using threshold
               selection = SelectFromModel(model, threshold=thresh, prefit=True)
               select_X_train = selection.transform(X_train)
               # train model
                selection_model = XGBClassifier()
                selection_model.fit(select_X_train, y_train)
                # eval model
                select_X_test = selection.transform(X_test)
                predictions = selection_model.predict(select_X_test)
                accuracy = accuracy_score(y_test, predictions)
                print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))